In [22]:
import os
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

In [ ]:
# Đọc tệp CSV đã tải lên
file_path = 'preProcessedDataWithHmdx.csv'
data = pd.read_csv(file_path)

# Hiển thị thông tin tổng quan về dữ liệu
data = data.drop(columns=['HMDX_label', 'PrecipAmount_sum'])

data.info(), data.head()

In [ ]:
# Tính ma trận tương quan
correlation_matrix = data.corr()

# Xác định các cặp thuộc tính có tương quan lớn hơn 0.75
high_corr_pairs = correlation_matrix[correlation_matrix.abs() > 0.75].stack().reset_index()
high_corr_pairs = high_corr_pairs[high_corr_pairs['level_0'] != high_corr_pairs['level_1']]

# Lọc danh sách các cột cần loại bỏ
columns_to_drop = set()
for col1, col2, corr in high_corr_pairs.itertuples(index=False):
    # Chỉ giữ lại một cột trong mỗi cặp
    if col1 not in columns_to_drop:
        columns_to_drop.add(col2)

# Loại bỏ các cột có tương quan cao
filtered_data = data.drop(columns=list(columns_to_drop), axis=1)
filtered_data.info()

In [25]:
# Kết hợp các cột Year, Month, Day thành một cột Date
filtered_data['Date'] = pd.to_datetime(filtered_data[['Year', 'Month', 'Day']])

# Đặt cột 'Date' làm chỉ số thời gian
filtered_data.set_index('Date', inplace=True)

# Xóa các cột không cần thiết
filtered_data.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

train_data = filtered_data.iloc[:-30]['Temp_mean']
test_data = filtered_data.iloc[-30:]['Temp_mean']

In [ ]:
result = adfuller(train_data)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
plot_acf(train_data)
plot_pacf(train_data)
plt.show()

In [ ]:
# Tạo thư mục lưu đồ thị nếu chưa tồn tại
output_dir = "../arima_forecasts"
os.makedirs(output_dir, exist_ok=True)

# Vòng lặp thử nghiệm các giá trị p, d, q
for p in range(8):
    for d in range(8):
        for q in range(8):
            try:
                # Xây dựng và huấn luyện mô hình ARIMA
                model = ARIMA(train_data, order=(p, d, q))
                model_fit = model.fit()

                # Dự đoán
                forecast = model_fit.forecast(steps=30)

                # Vẽ đồ thị
                plt.figure(figsize=(12, 6))
                plt.plot(test_data.index, test_data, label='Actual Data (Last 30 Days)', color='blue')
                plt.plot(test_data.index, forecast, label='Forecast', color='red', linestyle='--')
                plt.xlabel('Date')
                plt.ylabel('Temperature Mean')
                plt.title(f'ARIMA Forecast (p={p}, d={d}, q={q})')
                plt.legend()
                plt.grid()

                # Lưu đồ thị
                file_name = f"arima_p{p}_d{d}_q{q}.png"
                plt.savefig(os.path.join(output_dir, file_name))
                plt.close()
            except Exception as e:
                print(f"Failed for ARIMA(p={p}, d={d}, q={q}): {e}")


In [ ]:
p, d, q = 2, 4, 5
model = ARIMA(train_data, order=(p, d, q))
model_fit = model.fit()

# Dự đoán
forecast = model_fit.forecast(steps=30)

# Vẽ đồ thị
plt.figure(figsize=(12, 6))
plt.plot(test_data.index, test_data, label='Actual Data (Last 30 Days)', color='blue')
plt.plot(test_data.index, forecast, label='Forecast', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Temperature Mean')
plt.title(f'ARIMA Forecast (p={p}, d={d}, q={q})')
plt.legend()
plt.show()